In [11]:
# pip install fsspec / conda install -c conda-forge fsspec
# pip install s3fs
import pandas as pd
import numpy as np
import boto3
import os
from sagemaker import get_execution_role

bucket_name = 'eliezerraj-908671954593-dataset'
prefix_name = 'payment'
file_name = 'payment_1503.csv'

data_location = 's3://{}/{}/{}'.format(bucket_name, prefix_name, file_name)

role = get_execution_role()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [12]:
df = pd.read_csv(data_location)
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
df.head(5)

,id,fk_card_id,card_number,terminal_name,coord_x,coord_y,card_type,card_model,payment_at,mcc,amount,night_day,ic_night_day,wkend_wkday,ic_wkend_wkday,day_of_year,tx_1d,avg_1d,tx_7d,avg_7d,tx_30d,avg_30d,time_btw_tx,time_btw_cc_tx,fraud
0,1,54,111.111.000.054,TERM-147,120.0,182.0,CREDIT,VIRTUAL,2024-01-01 09:00:19.000 -0300,CINEMA,178.0,day,0,wkday,0,1,4,413.5,4,413.5,4,413.5,40.0,0.0,1.0
1,2,54,111.111.000.054,TERM-140,144.0,110.0,CREDIT,VIRTUAL,2024-01-01 09:00:59.000 -0300,COSMETIC,512.0,day,0,wkday,0,1,4,413.5,4,413.5,4,413.5,36.0,40.0,1.0
2,3,100,111.111.000.100,TERM-164,197.0,104.0,CREDIT,VIRTUAL,2024-01-01 09:01:35.000 -0300,COSMETIC,440.0,day,0,wkday,0,1,3,390.0,3,390.0,3,390.0,72.0,0.0,1.0
3,4,54,111.111.000.054,TERM-146,111.0,159.0,CREDIT,VIRTUAL,2024-01-01 09:02:47.000 -0300,DRUG_STORE,468.0,day,0,wkday,0,1,4,413.5,4,413.5,4,413.5,38.0,108.0,1.0
4,5,100,111.111.000.100,TERM-76,3.0,81.0,CREDIT,VIRTUAL,2024-01-01 09:03:25.000 -0300,LAUNDRY,228.0,day,0,wkday,0,1,3,390.0,3,390.0,3,390.0,40.0,110.0,1.0


In [13]:
train_df, validation_df, test_df= np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df)), int(0.9 * len(df))])

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [14]:
train_df.to_csv('train.csv', index=False, header=False)
validation_df.to_csv('validation.csv', index=False, header=False)
test_df.to_csv('test.csv', index=False, header=False)

In [15]:
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix_name, 'test/test_df.csv')).upload_file('validation.csv')